In [1]:
import requests
from bs4 import BeautifulSoup

def get_soup(url):
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',

              'Accept': 'text/html',

              'Referer': 'https://www.example.com/',

              'Cache-Control': 'no-cache'}
    try:
        resp=requests.get(url, headers=headers)
        resp.encoding='utf-8'
        if resp.status_code==200:
            soup=BeautifulSoup(resp.text, 'lxml')
            return soup
        else:
            print('網頁取得失敗', resp.status_code)
    except Exception as e:
        print('網址錯誤', e)


### 目標
- 首頁：本週排名、上週排名、片名、連結
- 子頁：上映年、片長、類型、IMDb Rating、預告片連結、圖片連結

In [97]:
# 測試以上程式碼，發現若有問題，都是在「片長」之後才出現問題，檢查結果，是「片長」該處有時沒有「分級資訊」，所以會找不到[2]，所以要用條件式判斷
# 先試片長改寫在有問題的第23名電影看可行否

subsoup=get_soup('https://www.imdb.com/title/tt15654328/?ref_=chtmvm_t_23')
ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
leng=ul.find_all('li',class_='ipc-inline-list__item')
if len(leng)==3:
    length=leng[2].text.strip()
else:
    length=leng[1].text.strip()
print(length)

2h 44m


In [115]:
pop_movies = []

for i in range(0,100):   #抓第1~100名的電影資訊
    
# 把原來的[0]都換成[i]；中間都print出訊息，才知道進度
# 加try, except，才不會因錯誤而中斷
    movie=[]

    try:

        print(f'現在開始抓第{i+1}名的電影...')

        # 本周排名
        print(f'現在開始抓第{i+1}名的目前排名！')
        cRank=data[i]['currentRank']
        print(cRank)

        # 上周排名
        print(f'現在開始抓第{i+1}名之前的排名！')
        if data[i]['node']['meterRanking']['rankChange']['changeDirection']=='UP':
            pRank=cRank+data[i]['node']['meterRanking']['rankChange']['difference']
        elif data[i]['node']['meterRanking']['rankChange']['changeDirection']=='DOWN':
            pRank=cRank-data[i]['node']['meterRanking']['rankChange']['difference']
        else:
            pRank=cRank
        print(pRank)

        #片名
        print(f'現在開始抓第{i+1}名的片名！')
        title = data[i]['node']['originalTitleText']['text']
        print(title)

        #連結
        print(f'現在開始抓第{i+1}名的連結！')
        num= data[i]['node']['id']
        num
        link = 'https://imdb.com/title/'+num
        print(link)

        # 入去囝頁的兩項用link入去，愛佇遮處理
        subsoup=get_soup(link)
        # 片長
        # 發現若有問題，攏是佇「片長」了後有問題，檢查結果，是「片長」彼位有時無「分級資訊」，所以會揣無[2]，愛用條件式判斷
        print(f'現在開始抓第{i+1}名的片長！')
        ul=subsoup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
        leng=ul.find_all('li',class_='ipc-inline-list__item')
        if len(leng)==3:
            length=leng[2].text.strip()
        else:
            length='None'
        print(length)

        # 預告片連結
        print(f'現在開始抓第{i+1}名的預告片連結！')
        prem=subsoup.find('a',class_='ipc-lockup-overlay ipc-focusable').get('href')
        premlink = 'https://imdb.com'+prem
        print(premlink)

        #上映年
        print(f'現在開始抓第{i+1}名的發行年度！')
        year = data[i]['node']['releaseYear']['year']
        print(year)


        #IMDb Rating
        print(f'現在開始抓第{i+1}名的評分！')
        rating=data[i]['node']['ratingsSummary']['aggregateRating']
        print(rating)

        #類型
        print(f'現在開始抓第{i+1}名的類型！')
        genre=data[i]['node']['titleGenres']['genres'][0]['genre']['text']
        print(genre)


        #圖片連結
        print(f'現在開始抓第{i+1}名的圖片連結！')
        pic=genre=data[i]['node']['primaryImage']['url']
        print(pic)
        

        pop_movies.append({
        'current ranking': cRank,
        'previous ranking': pRank,
        'title': title,
        'link': link,
        'length': length,
        'trailer': premlink,
        'year': year,
        'IMDb rating': rating,
        'genre': data[i]['node']['titleGenres']['genres'][0]['genre']['text'],
        'poster': pic
        })

    except Exception as e:
        print(f'第{i+1}名有問題！',e)
        pass
    
pop_movies.append(movie)

現在開始抓第1名的電影...
現在開始抓第1名的目前排名！
1
現在開始抓第1名之前的排名！
1
現在開始抓第1名的片名！
Five Nights at Freddy's
現在開始抓第1名的連結！
https://imdb.com/title/tt4589218
現在開始抓第1名的片長！
1h 49m
現在開始抓第1名的預告片連結！
https://imdb.com/title/tt4589218/mediaviewer/rm1384260097/?ref_=tt_ov_i
現在開始抓第1名的發行年度！
2023
現在開始抓第1名的評分！
5.5
現在開始抓第1名的類型！
Horror
現在開始抓第1名的圖片連結！
https://m.media-amazon.com/images/M/MV5BNWI3ZmY4NmItMGQ4My00ODJlLWJlNTktYjk2NzRkODU3YTNlXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_.jpg
現在開始抓第2名的電影...
現在開始抓第2名的目前排名！
2
現在開始抓第2名之前的排名！
2
現在開始抓第2名的片名！
Killers of the Flower Moon
現在開始抓第2名的連結！
https://imdb.com/title/tt5537002
現在開始抓第2名的片長！
3h 26m
現在開始抓第2名的預告片連結！
https://imdb.com/title/tt5537002/mediaviewer/rm229793537/?ref_=tt_ov_i
現在開始抓第2名的發行年度！
2023
現在開始抓第2名的評分！
8
現在開始抓第2名的類型！
Crime
現在開始抓第2名的圖片連結！
https://m.media-amazon.com/images/M/MV5BNjQwOGM2YTItMGYwNC00YTM4LWI0Y2QtZjQ2ZDcyMmRjMTFhXkEyXkFqcGdeQXVyMDM2NDM2MQ@@._V1_.jpg
現在開始抓第3名的電影...
現在開始抓第3名的目前排名！
3
現在開始抓第3名之前的排名！
6
現在開始抓第3名的片名！
Pain Hustlers
現在開始抓第3名的連結！
https://imdb.com/title/tt15257160
現在

In [116]:
import pandas as pd

    # 建立Pandas DataFrame
df = pd.DataFrame(pop_movies, columns=['current ranking', 'previous ranking', 'title', 'link', 'length', 'trailer', 'year', 'IMDb rating', 'genre', 'poster'])
df

,current ranking,previous ranking,title,link,length,trailer,year,IMDb rating,genre,poster
0,1.0,1.0,Five Nights at Freddy's,https://imdb.com/title/tt4589218,1h 49m,https://imdb.com/title/tt4589218/mediaviewer/r...,2023.0,5.5,Horror,https://m.media-amazon.com/images/M/MV5BNWI3Zm...
1,2.0,2.0,Killers of the Flower Moon,https://imdb.com/title/tt5537002,3h 26m,https://imdb.com/title/tt5537002/mediaviewer/r...,2023.0,8.0,Crime,https://m.media-amazon.com/images/M/MV5BNjQwOG...
2,3.0,6.0,Pain Hustlers,https://imdb.com/title/tt15257160,2h 2m,https://imdb.com/title/tt15257160/mediaviewer/...,2023.0,6.5,Crime,https://m.media-amazon.com/images/M/MV5BNWMxYj...
3,4.0,47.0,A Haunting in Venice,https://imdb.com/title/tt22687790,1h 43m,https://imdb.com/title/tt22687790/mediaviewer/...,2023.0,6.6,Crime,https://m.media-amazon.com/images/M/MV5BYmI4Mz...
4,5.0,15.0,The Killer,https://imdb.com/title/tt1136617,1h 58m,https://imdb.com/title/tt1136617/mediaviewer/r...,2023.0,7.4,Action,https://m.media-amazon.com/images/M/MV5BZGJkMD...
...,...,...,...,...,...,...,...,...,...,...
95,97.0,63.0,Winnie-the-Pooh: Blood and Honey,https://imdb.com/title/tt19623240,1h 24m,https://imdb.com/title/tt19623240/mediaviewer/...,2023.0,2.9,Horror,https://m.media-amazon.com/images/M/MV5BNGMwNj...
96,98.0,180.0,Terrifier,https://imdb.com/title/tt4281724,1h 25m,https://imdb.com/title/tt4281724/mediaviewer/r...,2016.0,5.6,Horror,https://m.media-amazon.com/images/M/MV5BYmMxNz...
97,99.0,90.0,A Nightmare on Elm Street,https://imdb.com/title/tt0087800,1h 31m,https://imdb.com/title/tt0087800/mediaviewer/r...,1984.0,7.4,Horror,https://m.media-amazon.com/images/M/MV5BNzFjZm...
98,100.0,89.0,Doctor Sleep,https://imdb.com/title/tt5606664,2h 32m,https://imdb.com/title/tt5606664/mediaviewer/r...,2019.0,7.3,Drama,https://m.media-amazon.com/images/M/MV5BMTRkYT...


In [118]:
# 用今仔日的日期來寄檔名

from datetime import datetime

date=datetime.now().strftime('%Y-%m-%d')  #m愛細本字，因為大本字是指分鐘
date

df.to_csv(f'IMDb電影排行榜_{date}.csv',encoding='utf-8-sig')